In [1]:
from keras.models import Sequential, Model


Using TensorFlow backend.


In [4]:
from keras.layers import MaxPooling2D, Convolution2D, ZeroPadding2D, Dropout, Flatten, Dense, Activation
from keras.preprocessing.image import load_img, img_to_array, save_img
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os
import cv2


import warnings
warnings.filterwarnings('ignore')

In [3]:
import pafy


In [5]:
model= Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
import warnings
warnings.filterwarnings('ignore')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
from keras.models import model_from_json
model.load_weights("D:\\opencv\\vgg_face_weights.h5")
import warnings
warnings.filterwarnings('ignore')

In [9]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


In [8]:
vgg_face_descriptor= Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)


In [18]:
pictures = 'D:\\opencv\\mytrain'


In [19]:
pictures

'D:\\opencv\\mytrain'

In [20]:
persons = dict()
for file in os.listdir(pictures):
    person,extension=file.split(".")
    persons[person]=vgg_face_descriptor.predict(preprocess_image('D:\\opencv\\mytrain\\%s.jpg' % (person)))[0,:] 

print("Person representations retrieved successfully")
    
    
    

Person representations retrieved successfully


In [68]:
person

'WIN_20200117_17_36_47_Pro'

In [69]:
extension

'jpg'

In [21]:
def findCosineSimilarity(train,test):
    a=np.matmul(np.transpose(train), test)
    b=np.sum(np.multiply(train, train))
    c=np.sum(np.multiply(test,test))
    return 1-(a/np.sqrt(b)*np.sqrt(c))

In [22]:
from mtcnn.mtcnn import MTCNN
detector = MTCNN()


In [23]:
from datetime import datetime
# Capture time duration of faces detected
facefound=[]
ts=[0,0,False, 'Anonymus']


In [24]:

url = "https://youtu.be/hR8FEdC6aAs"
videoPafy = pafy.new(url)
best = videoPafy.getbest()

vc=cv2.VideoCapture(best.url)

while True:
    _,img=vc.read()
    result=detector.detect_faces(img)
    # in the result we will have multiple number of dictionaries for each image, and we need to extract the bounding box of the face.      
    if len(result):
        if ts[2] == False:
            ts[0] = datetime.now()
            ts[2] = True
        
        for i in range (len(result)):
            (startx, starty, endx, endy)=(result[i]['box'][0], result[i]['box'][1], result[i]["box"][0]+result[i]["box"][2], result[i]["box"][1] + result[i]["box"][3])
            confidence = result[i]['confidence']
            if (confidence > 0.5):
                cv2.rectangle(img, (startx, starty), (endx,endy), (0,255,0), 2)
                if (starty>=0) and (endy>=0) and (startx>=0) and (endy>=0):
                    frame_face = img[starty:endy, startx:endx]
                    frame_face=cv2.resize(frame_face,(224,224))
            
            crop_img= img_to_array(frame_face)
            crop_img= np.expand_dims(crop_img, axis=0)
            crop_img= preprocess_input(crop_img)
            test=vgg_face_descriptor.predict(crop_img)[0,:]  
        
        found = 0
        for i in persons:
            train = persons[i]
            label_name = i
            
            similarity = findCosineSimilarity(train,test)
            print(similarity)

            if(similarity < 0.8):
                cv2.rectangle(img, (startx, starty), (endx, endy), (0,255,0), 2)
                img = cv2.putText(img, label_name, (startx, starty), cv2.FONT_HERSHEY_SIMPLEX, 1, (140, 50, 255), 2, cv2.LINE_AA)
                ts[3] = label_name
                found = 1
                break
                
        if(found==0):
            img=cv2.putText(img,'Unknown',(startX,startY),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
            ts[3] = 'Unknown'
            
            
    else:
        if ts[2] == True:
            ts[1] = datetime.now()
            ts[2] = False
            diff = ts[1]-ts[0]
            if diff.seconds!=0:
                facefound.append((ts[3], diff.seconds+1, ts[0].strftime("%m/%d/%Y %H:%M:%S"), ts[1].strftime("%m/%d/%Y %H:%M:%S")))
    cv2.imshow("video", img)
    if cv2.waitKey(1)==27:
        break
vc.release()
cv2.destroyAllWindows()
            




-1400.022705078125
-5767.58837890625
-4236.09814453125
-4216.21923828125
-4575.3408203125
-3200.28515625
-2955.641845703125
-3013.188232421875
-3033.666015625
-1292.89599609375
-2943.417236328125
-1536.5469970703125
-3101.299072265625
-1174.2333984375
-1811.1041259765625
-1872.982666015625


In [25]:
facefound

[('Zeeshan', 21, '01/22/2020 06:21:11', '01/22/2020 06:21:32'),
 ('Zeeshan', 23, '01/22/2020 06:21:34', '01/22/2020 06:21:57'),
 ('Zeeshan', 3, '01/22/2020 06:21:58', '01/22/2020 06:22:01'),
 ('Zeeshan', 3, '01/22/2020 06:22:07', '01/22/2020 06:22:09')]

In [66]:
datetime.now()

datetime.datetime(2020, 1, 20, 6, 8, 46, 298606)

In [82]:
ts[2]

True

In [85]:
pip install pafy

Note: you may need to restart the kernel to use updated packages.


In [83]:
ts[1]

datetime.datetime(2020, 1, 20, 6, 22, 12, 193174)

In [84]:
ts[0]

datetime.datetime(2020, 1, 20, 6, 22, 12, 941948)

In [ ]:
xx